# Data Engineering Usecase

In [69]:
#import the necessary libraries
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime as dt

In [68]:
pwd

'C:\\Users\\DELL\\Documents'

### Load CSV files in pandas data frame 

In [105]:
date_df=pd.read_csv("date_master.csv",na_values=['nan'])
date_df.head(1)

,Year,Quarter,MonthNo,MonthName,MonthID,Date
0,1999.0,Q1,1.0,Jan,199901.0,1999-01-01


In [79]:
with open('us_sales_2000_to_2016.csv') as f:
    for_pd = StringIO()
    for line in f:
        new_line = re.sub(r',', '|', line.rstrip(), count=5)
        print(new_line, file=for_pd)
    for_pd.seek(0)

sales_df=pd.read_csv(for_pd, sep='|',error_bad_lines=False)
sales_df.head(1)

C:\Users\DELL\AppData\Local\Temp\ipykernel_7036\3398542620.py:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  sales_df=pd.read_csv(for_pd, sep='|',error_bad_lines=False)
b'Skipping line 10439374: expected 5 fields, saw 6\nSkipping line 10439381: expected 5 fields, saw 6\n'


,ProductID,Date,Zip,Units,Revenue
0,833,2000-01-15,76108,1,797.9475


In [97]:
with open('product_master.csv') as f:
    for_pd = StringIO()
    for line in f:
        new_line = re.sub(r',', '|', line.rstrip(), count=5)
        print(new_line, file=for_pd)
    for_pd.seek(0)

product_df=pd.read_csv(for_pd, sep='|')
product_df.head(1)

,ï»¿ProductID,Product,Category,Segment,ManufacturerID,Manufacturer
0,1,Abbas MA-01,Mix,All Season,1,Abbas


In [147]:
with open('zip_usa_master.csv') as f:
    for_pd = StringIO()
    for line in f:
        new_line = re.sub(r',', '|', line.rstrip(), count=11)
        print(new_line, file=for_pd)
    for_pd.seek(0)

zip_df=pd.read_csv(for_pd, sep='|')
zip_df.head(1)

,CountryISOCode,PostalCode,PlaceName,AdminName1,CodeName1,AdminName2,CodeName2,AdminName3,CodeName3,latitude,longitude,accuracy
0,US,99553,Akutan,Alaska,AK,Aleutians East,13.0,NaN,NaN,54.143,-165.7854,1


### Data preprocessing for dataframes 

#### Date dataframe 

In [110]:
# Pre processing the date dataframe to populate data from 1999 to 2016
date_df['Year'] = date_df['Year'].fillna(0).astype(int)
date_df['MonthNo'] = date_df['MonthNo'].fillna(0).astype(int)
date_df['MonthID'] = date_df['MonthID'].fillna(0).astype(int)
date_df.head(5)

,Year,Quarter,MonthNo,MonthName,MonthID,Date,Dummy
0,1999,Q1,1,Jan,199901,1999-01-01,1999Q1
1,1999,Q1,0,NaN,0,1999-01-02,1999Q1
2,1999,Q1,0,NaN,0,1999-01-03,1999Q1
3,1999,Q1,0,NaN,0,1999-01-04,1999Q1
4,1999,Q1,0,NaN,0,1999-01-05,1999Q1


In [100]:
date_df['Date'] = pd.to_datetime(date_df['Date'])
date_df['Date'] = pd.to_datetime(date_df['Date']).dt.strftime('%Y-%m-%d')
date_df['Date']

In [111]:
date_df['Dummy'] = pd.PeriodIndex(date_df["Date"], freq='Q')
date_df['Quarter']=date_df['Dummy'].astype(str).str[4:]
date_df['Year']=date_df['Dummy'].astype(str).str[:4]
date_df['MonthNo']=date_df['Date'].astype(str).str[5:7]

In [119]:
date_df.drop(columns=['MonthName','MonthID','Dummy'], inplace=True)

In [155]:
date_df.head(5)

,Year,Quarter,MonthNo,Date
0,1999,Q1,01,1999-01-01
1,1999,Q1,01,1999-01-02
2,1999,Q1,01,1999-01-03
3,1999,Q1,01,1999-01-04
4,1999,Q1,01,1999-01-05


#### Sales dataframe 

In [128]:
sales_df.shape

(10439384, 6)

In [136]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10439384 entries, 0 to 10439383
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   ProductID  int64  
 1   Date       object 
 2   Zip        int64  
 3   Units      int64  
 4   Revenue    float64
 5   Year       object 
 6   Month      object 
dtypes: float64(1), int64(3), object(3)
memory usage: 557.5+ MB


In [130]:
sales_df['Year']=sales_df['Date'].astype(str).str[:4]
sales_df['Month']=sales_df['Date'].astype(str).str[5:7]

In [164]:
sales_df['Revenue'] = sales_df['Revenue'].round(1)

In [165]:
sales_df.head()

,ProductID,Date,Zip,Units,Revenue,Year,Month
0,833,2000-01-15,76108,1,798.0,2000,01
1,837,2000-01-15,33436,1,814.0,2000,01
2,837,2000-01-15,76531,1,814.0,2000,01
3,838,2000-01-15,29526,1,934.0,2000,01
4,838,2000-01-15,33584,1,945.0,2000,01


#### Product dataframe  

In [134]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2412 entries, 0 to 2411
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ï»¿ProductID    2412 non-null   int64 
 1   Product         2412 non-null   object
 2   Category        2410 non-null   object
 3   Segment         2412 non-null   object
 4   ManufacturerID  2412 non-null   object
 5   Manufacturer    2412 non-null   object
dtypes: int64(1), object(5)
memory usage: 113.2+ KB


In [138]:
product_df.head(2)


,ï»¿ProductID,Product,Category,Segment,ManufacturerID,Manufacturer
0,1,Abbas MA-01,Mix,All Season,1,Abbas
1,2,Abbas MA-02,Mix,All Season,1,Abbas


#### ZIP dataframe  

In [148]:
zip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41469 entries, 0 to 41468
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CountryISOCode  41469 non-null  object 
 1   PostalCode      41469 non-null  int64  
 2   PlaceName       41469 non-null  object 
 3   AdminName1      40956 non-null  object 
 4   CodeName1       40958 non-null  object 
 5   AdminName2      40958 non-null  object 
 6   CodeName2       40958 non-null  float64
 7   AdminName3      1 non-null      object 
 8   CodeName3       1 non-null      float64
 9   latitude        41467 non-null  float64
 10  longitude       41469 non-null  float64
 11  accuracy        40939 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 3.8+ MB


In [149]:
zip_df.drop(['AdminName3', 'CodeName3'], axis=1,inplace=True)

In [150]:
zip_df.head(5)

,CountryISOCode,PostalCode,PlaceName,AdminName1,CodeName1,AdminName2,CodeName2,latitude,longitude,accuracy
0,US,99553,Akutan,Alaska,AK,Aleutians East,13.0,54.1430,-165.7854,1
1,US,99571,Cold Bay,Alaska,AK,Aleutians East,13.0,55.1858,-162.7211,1
2,US,99583,False Pass,Alaska,AK,Aleutians East,13.0,54.8542,-163.4113,1
3,US,99612,King Cove,Alaska,AK,Aleutians East,13.0,55.0628,-162.3056,1
4,US,99661,Sand Point,Alaska,AK,Aleutians East,13.0,55.3192,-160.4914,1


In [151]:
date_df.to_csv('dim_date.csv')

product_df.to_csv('dim_product.csv')
zip_df.to_csv('dim_zip.csv')

In [160]:
#sales_df.to_csv('fact_sales.csv')

csvfile = open('fact_sales.csv', 'r').readlines()
filename = 1
for i in range(len(csvfile)):
    if i % 3000000 == 0:
        open(str(filename) + '.csv', 'w+').writelines(csvfile[i:i+3000000])
        filename += 1

In [166]:
pip install snowflake-connector-python


  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1


In [171]:
import snowflake.connector


    # Connect to snowflake
ctx = snowflake.connector.connect(
user='spilli',
password='*%TuMbHh5Z&FcUC8',
account='mvb03398.us-east-1'

    # Establish a cursor and execute a query
 cs = ctx.cursor()
    ret = []
        try:
            test = cs.execute("USE database SNOWFLAKE_SAMPLE_DATA")
            cs.execute("SHOW TABLES")
            ret = cs.fetchmany(30)
        finally:
        # Always close connections if there is an error
            cs.close()

    # Close the connection
ctx.close()

    # Print out the return values for the data
for data in ret:
    print(data)
        
        

SyntaxError: invalid syntax (2907854865.py, line 11)